In [1]:
# Imports

import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import pickle 
import numpy as np
import matplotlib.pyplot as plt


from sklearn.ensemble import RandomForestClassifier

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 40)

In [2]:
# Load the npy file produced in the preprocessing notebook
with open('../../Data collection/ProPlay/game_data.pkl', 'rb') as f:
    game_data = pickle.load(f)

In [3]:
def extract_gamestats(game_number):
    '''
    Extracts game data from a single game which consists of a dictionary with keys 'metadata' and 'data'.
    
    '''
    game = game_data[game_number]
    
    game_stats = {}
    
    metadata = game['metadata']
    data = game['data']
    
    game_stats['GAME_NUMBER'] = game_number
    game_stats['DATE'] = metadata['DATE']
    game_stats['PATCH'] = metadata['PATCH']
    game_stats['BLUE_TEAM'] = metadata['BLUE_TEAM']
    game_stats['RED_TEAM'] = metadata['RED_TEAM']
    game_stats['GAME_WINNER'] = metadata['GAME_WINNER']
    game_stats['MATCH_WINNER'] = metadata['MATCH_WINNER']
    
    game_stats['games_in_match'] = metadata['GAMES_IN_MATCH']
    game_stats['MATCH_SCORE'] = metadata['MATCH_SCORE']
    
    game_stats['GD'] = data['Golds'][0:5].sum() - data['Golds'][5:].sum()
    game_stats['GD@15'] = data['GD@15'][0:5].sum()
    
    game_stats['GPMD'] = data['GPM'][0:5].sum() - data['GPM'][5:].sum()
    game_stats['CSMD'] = data['CSM'][0:5].sum() - data['CSM'][5:].sum()
    

    
    return game_stats
    

In [4]:
game_numbers = list(game_data.keys())

In [5]:
%%time
df_dict = [extract_gamestats(num) for num in game_numbers]

df =  pd.DataFrame(df_dict)
df = df.set_index('GAME_NUMBER')

Wall time: 439 ms


In [7]:
df

,DATE,PATCH,BLUE_TEAM,RED_TEAM,GAME_WINNER,MATCH_WINNER,games_in_match,MATCH_SCORE,GD,GD@15,GPMD,CSMD
GAME_NUMBER,,,,,,,,,,,,
257,2014-10-19,4.14,Star Horn Royal Club,Samsung Galaxy White,Samsung Galaxy White,Samsung Galaxy White,4,"[1, 3]",-18728.0,-4884.0,-764.0,-2.5
258,2014-10-19,4.14,Samsung Galaxy White,Star Horn Royal Club,Samsung Galaxy White,Samsung Galaxy White,4,"[1, 3]",19765.0,2766.0,676.0,3.2
259,2014-10-19,4.14,Star Horn Royal Club,Samsung Galaxy White,Star Horn Royal Club,Samsung Galaxy White,4,"[1, 3]",10371.0,2182.0,269.0,0.5
260,2014-10-19,4.14,Samsung Galaxy White,Star Horn Royal Club,Samsung Galaxy White,Samsung Galaxy White,4,"[1, 3]",15523.0,2011.0,675.0,4.2
252,2014-10-12,4.14,Star Horn Royal Club,OMG,OMG,Star Horn Royal Club,5,"[3, 2]",-21841.0,-4325.0,-743.0,-4.0
...,...,...,...,...,...,...,...,...,...,...,...,...
44341,2022-10-07,12.18,T1,Edward Gaming,T1,T1,1,"[1, 0]",8802.0,2932.0,385.0,4.0
44340,2022-10-07,12.18,JD Gaming,Evil Geniuses,JD Gaming,JD Gaming,1,"[1, 0]",10842.0,2377.0,364.0,2.1
44339,2022-10-07,12.18,CTBC Flying Oyster,100 Thieves,CTBC Flying Oyster,CTBC Flying Oyster,1,"[1, 0]",9693.0,2338.0,304.0,2.3
